In [19]:
%run Imports.ipynb
from quantpy.tomography.interval import ConfidenceInterval, _pop_hidden_keys, Mode

import numpy as np
import scipy.linalg as la
import scipy.stats as sts
import polytope as pc
import math
import pypoman

from enum import Enum, auto
from abc import ABC, abstractmethod
from scipy.interpolate import interp1d
from collections import Counter, defaultdict
from functools import partial
from cvxopt import matrix, solvers
from einops import repeat, rearrange

from quantpy.geometry import hs_dst, trace_dst, if_dst
from quantpy.polytope import compute_polytope_volume, find_max_distance_to_polytope
from quantpy.qobj import Qobj
from quantpy.routines import (
    _left_inv, _vec2mat, _mat2vec,
    _matrix_to_real_tril_vec, _real_tril_vec_to_matrix, generate_pauli, generate_single_entries
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
class ProcessWangIntervalFidelity(ConfidenceInterval):
    def __init__(self, tmg, n_points=1000, target_channel=None):
        kwargs = _pop_hidden_keys(locals())
        super().__init__(tmg, **kwargs)
        
    def __call__(self, conf_levels=None):
        if conf_levels is None:
            conf_levels = np.linspace(1e-3, 1-1e-3, 1000)
        if not hasattr(self, 'cl_to_dist_max'):
            self.setup()
        return (self.cl_to_dist_min(conf_levels), self.cl_to_dist_max(conf_levels)), conf_levels
        
    def setup(self):
        channel = self.tmg.channel
        dim_in = dim_out = 2 ** channel.n_qubits
        dim = dim_in * dim_out
        bloch_indices = [i for i in range(dim ** 2) if i % (dim_out ** 2) != 0]
        
        if self.target_channel is None:
            self.target_channel = channel
        
        povm_matrix = self.tmg.tomographs[0].povm_matrix
        n_measurements = self.tmg.tomographs[0].n_measurements

        frequencies = np.asarray([
            np.clip(tmg.raw_results / tmg.n_measurements[:, None], self.EPS, 1 - self.EPS)
            for tmg in self.tmg.tomographs
        ])
        
        meas_matrix = (np.reshape(povm_matrix * n_measurements[:, None, None] / np.sum(n_measurements),
                                  (-1, povm_matrix.shape[-1])) 
                       * povm_matrix.shape[0])
        states_matrix = np.asarray([rho.T.bloch for rho in self.tmg.input_basis.elements])
        channel_matrix = np.einsum("i a, j b -> i j a b", states_matrix, meas_matrix[:, 1:]) * dim
        channel_matrix = rearrange(channel_matrix, "i j a b -> (i j) (a b)")
        A = np.ascontiguousarray(channel_matrix)
        
        max_delta = count_delta(1-1e-5, frequencies, n_measurements)
        min_delta = count_delta(0, frequencies, n_measurements)
        deltas = np.linspace(min_delta, max_delta, self.n_points)
        
        dist_max = []
        dist_min = []
        for delta in deltas:
            b = (np.clip(np.hstack(np.concatenate(frequencies, axis=0)) + delta, self.EPS, 1 - self.EPS) 
                 - repeat(meas_matrix[:, 0], 'a -> (a b)', b=len(states_matrix)))
            return A, b
            c = matrix(self.target_channel.choi.bloch[bloch_indices])
            G, h = matrix(A), matrix(b)
            sol = solvers.lp(c, G, h)
            if not sol['primal objective']:
                dist_min.append(1)
            else:
                dist_min.append(1/dim + sol['primal objective'])
            sol = solvers.lp(-c, G, h)
            if not sol['primal objective']:
                dist_max.append(1)
            else:
                dist_max.append(1/dim - sol['primal objective'])

        conf_levels = []
        for delta in deltas:
            conf_levels.append(count_confidence(delta, frequencies, self.tmg.tomographs[0].n_measurements))
        self.cl_to_dist_max = interp1d(conf_levels, dist_max)
        self.cl_to_dist_min = interp1d(conf_levels, dist_min)

In [30]:
channel = qp.operator.Id.as_channel()
tmg = qp.ProcessTomograph(channel, input_states='proj4')
tmg.experiment(10000)
channel_hat = tmg.point_estimate()
interval = ProcessWangIntervalFidelity(tmg)
A, b = interval.setup()

NameError: name 'count_delta' is not defined

In [28]:
channel.choi.bloch

array([ 0.5,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5,
        0. ,  0. ,  0. ,  0. ,  0.5])

In [50]:
def zero_channel(p=1):
    return qp.Channel(lambda rho: rho * (1 - p) + qp.Qobj([0, 0, 0.5]) * p * rho.trace(), n_qubits=1)

In [43]:
[channel.transform(e) for e in se]

[Quantum object
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]),
 Quantum object
 array([[0. +0.j, 0.4+0.j],
        [0. +0.j, 0. +0.j]]),
 Quantum object
 array([[0. +0.j, 0. +0.j],
        [0.4+0.j, 0. +0.j]]),
 Quantum object
 array([[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]])]

In [15]:
x = np.zeros(12)
x[[0,]] = -1
A @ x

array([-1., -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [16]:
channel.choi.bloch

array([ 0.5,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5,
        0. ,  0. ,  0. ,  0. ,  0.5])

In [18]:
x = np.zeros(16)
x[0] = 0.5
x[4] = -0.5

In [7]:
A

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  

In [5]:
interval = ProcessWangIntervalFidelity()

Quantum channel with Choi matrix
array([[ 0.99753+0.j     , -0.0044 -0.0015j ,  0.00277-0.00011j,
         0.99256-0.0039j ],
       [-0.0044 +0.0015j ,  0.00247+0.j     ,  0.00179+0.0007j ,
        -0.00277+0.00011j],
       [ 0.00277+0.00011j,  0.00179-0.0007j ,  0.00342+0.j     ,
         0.00235+0.00175j],
       [ 0.99256+0.0039j , -0.00277-0.00011j,  0.00235-0.00175j,
         0.99658+0.j     ]])